# Home Page

In [ ]:
import streamlit as st

# Page config
st.set_page_config(
    page_title="Cricbuzz LiveStats",
    page_icon="🏏",
    layout="wide"
)


# Main Header
st.title("🏏 Cricbuzz LiveStats")
st.subheader("Real-Time Cricket Insights & SQL-Based Analytics")

# Intro
st.info(
    "Welcome to Cricbuzz LiveStats! 🚀\n\n"
    "This platform integrates live data from the Cricbuzz API with SQL databases "
    "to deliver real-time match updates, player statistics, and analytics."
)

# Technologies
st.header("🛠️ Technologies Used")
col1, col2, col3 = st.columns(3)
with col1:
    st.markdown("- Streamlit\n- Plotly\n- Pandas")
with col2:
    st.markdown("- Python\n- REST API\n- JSON")
with col3:
    st.markdown("- MySQL\n- SQLite\n- PyMySQL")

# Features
st.header("⭐ Key Features")
features = {
    "📺 Live Match Updates": "Real-time scorecards and match details",
    "📊 Player Statistics": "Batting and bowling stats with history",
    "📈 SQL Analytics": "25+ queries for deeper insights",
    "🛠️ CRUD Operations": "Manage players & match data",
    "🎯 Fantasy Support": "Player form & performance tracking",
    "📱 Responsive": "Works on desktop, tablet, and mobile"
}
for title, desc in features.items():
    st.markdown(f"**{title}** — {desc}")

# Getting Started
st.header("🚀 Getting Started")
st.markdown("""
1. Use the sidebar for navigation  
2. Check **Live Matches** for ongoing games  
3. Explore **Player Stats**  
4. Run **SQL Analytics**  
5. Manage data with **CRUD Operations**
""")

# Project Structure
st.header("📁 Project Structure")
st.code("""
Cricbuzz-LiveStats/
├── main.py
├── utils/
│   ├── db_connection.py
│   ├── api_handler.py
│   └── data_processor.py
├── pages/
│   ├── live_matches.py
│   ├── player_stats.py
│   ├── sql_analytics.py
│   └── crud_operations.py
├── data/sample_data.db
├── requirements.txt
└── README.md
""")

# Business Use Cases
st.header("💼 Business Use Cases")
st.markdown("""
- Sports Media: Real-time updates for commentary  
- Fantasy Platforms: Player form analysis  
- Analytics Firms: Player evaluation  
- Education: Database teaching with real data  
- Betting & Prediction: Odds based on performance  
""")

# Footer
st.markdown("---")
st.caption("Built with ❤️ using Streamlit, Python & MySQL | © 2024 Cricbuzz LiveStats Project")


# Dashboard

In [ ]:
import streamlit as st
import requests

# API Config
BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
HEADERS = {
    "x-rapidapi-key": "ddbcfd40b9msh911004891824267p1640f9jsn038fe760a8e0",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

st.set_page_config(page_title="Cricbuzz Live Scoreboard", layout="wide")
st.title("🏏 Cricbuzz Live Cricket Dashboard")

# Step 1: Get all live matches
matches_url = f"{BASE_URL}/matches/v1/live"
response = requests.get(matches_url, headers=HEADERS)
data = response.json()

match_options = {}
if "typeMatches" in data:
    for match_type in data["typeMatches"]:
        for series in match_type["seriesMatches"]:
            if "seriesAdWrapper" in series:
                for match in series["seriesAdWrapper"]["matches"]:
                    if "matchInfo" in match:
                        info = match["matchInfo"]
                        match_id = info["matchId"]
                        title = f"{info['team1']['teamName']} vs {info['team2']['teamName']} ({info.get('matchDesc','')})"
                        match_options[title] = match_id

# Dropdown for selecting a match
selected_match = st.selectbox("Select a Live Match:", list(match_options.keys()))

if selected_match:
    match_id = match_options[selected_match]

    # Step 2: Fetch scoreboard for selected match
    score_url = f"{BASE_URL}/mcenter/v1/{match_id}/hscard"
    score_resp = requests.get(score_url, headers=HEADERS)
    score_data = score_resp.json()

    # Step 3: Display scoreboard
    st.subheader(f"📊 Scoreboard for {selected_match}")

    if "scoreCard" in score_data:
        for inning in score_data["scoreCard"]:
            bat_team = inning["batTeamDetails"]["batTeamName"]

            # ✅ Correct keys
            runs = inning.get("runs", 0)
            wickets = inning.get("wickets", 0)
            overs = inning.get("overs", "0")

            st.markdown(f"### 🏏 {bat_team} - {runs}/{wickets} in {overs} overs")

            # Batsmen
            if "batsmenData" in inning["batTeamDetails"]:
                st.write("**Batting:**")
                bat_table = []
                for b in inning["batTeamDetails"]["batsmenData"].values():
                    bat_table.append([b["batName"], b["runs"], b["balls"], b["fours"], b["sixes"], b["strikeRate"]])
                st.table(bat_table)

            # Bowlers
            if "bowlTeamDetails" in inning and "bowlersData" in inning["bowlTeamDetails"]:
                st.write("**Bowling:**")
                bowl_table = []
                for bowler in inning["bowlTeamDetails"]["bowlersData"].values():
                    bowl_table.append([bowler["bowlName"], bowler["overs"], bowler["maidens"], bowler["runs"], bowler["wickets"], bowler["economy"]])
                st.table(bowl_table)

    else:
        st.warning("⚠️ Scoreboard not available for this match yet.")


# Players

In [ ]:
import streamlit as st
import requests
import pandas as pd

# API Config
BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
HEADERS = {
    "x-rapidapi-key": "ddbcfd40b9msh911004891824267p1640f9jsn038fe760a8e0",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

st.set_page_config(page_title="Cricbuzz Player Search", layout="wide")
st.title("🏏 Cricbuzz Player Search")

# Search bar
player_name = st.text_input("🔍 Search Player", placeholder="Type player name...")

if player_name.strip():
    url = f"{BASE_URL}/stats/v1/player/search"
    params = {"plrN": player_name.lower()}  # ✅ case-insensitive search

    try:
        response = requests.get(url, headers=HEADERS, params=params)
        data = response.json()

        if "player" in data and data["player"]:
            st.subheader(f"Results for **{player_name.title()}**")

            for p in data["player"]:
                if player_name.lower() in p.get("name", "").lower():
                    st.markdown(f"### {p.get('name', 'Unknown')} ({p.get('teamName', 'N/A')})")
                    st.write(f"**Player ID:** {p.get('id', 'N/A')}")

                    # Button to fetch full profile
                    if st.button(f"📄 View Profile: {p.get('name')}"):
                        player_id = p.get("id")

                        # ----- Player Info -----
                        info_url = f"{BASE_URL}/stats/v1/player/{player_id}"
                        info_res = requests.get(info_url, headers=HEADERS).json()
                        clean_info = {k: v for k, v in info_res.items() if k not in ["image", "bio", "faceImageId"]}

                        st.subheader(clean_info.get("name", "Unknown"))
                        st.write(f"**Nickname:** {clean_info.get('nickName', 'N/A')}")
                        st.write(f"**Role:** {clean_info.get('role', 'N/A')}")
                        st.write(f"**Batting Style:** {clean_info.get('bat', 'N/A')}")
                        st.write(f"**Bowling Style:** {clean_info.get('bowl', 'N/A')}")
                        st.write(f"**Team:** {clean_info.get('intlTeam', 'N/A')}")
                        st.write(f"**Other Teams:** {clean_info.get('teams', 'N/A')}")
                        st.write(f"**DOB:** {clean_info.get('DoB', 'N/A')}")
                        st.write(f"**Height:** {clean_info.get('height', 'N/A')}")
                        st.write(f"**Birth Place:** {clean_info.get('birthPlace', 'N/A')}")

                        if "appIndex" in clean_info:
                            st.markdown(f"[🌐 Full Profile]({clean_info['appIndex'].get('webURL', '#')})")

                        # ----- Batting Stats -----
                        bat_url = f"{BASE_URL}/stats/v1/player/{player_id}/batting"
                        bat_res = requests.get(bat_url, headers=HEADERS).json()

                        if "values" in bat_res:
                            st.markdown("### 🏏 Batting Stats")
                            bat_headers = bat_res["headers"][1:]  # Skip ROWHEADER
                            bat_data = {row["values"][0]: row["values"][1:] for row in bat_res["values"]}
                            df_bat = pd.DataFrame(bat_data, index=bat_headers).T
                            st.dataframe(df_bat)

                        # ----- Bowling Stats -----
                        bowl_url = f"{BASE_URL}/stats/v1/player/{player_id}/bowling"
                        bowl_res = requests.get(bowl_url, headers=HEADERS).json()

                        if "values" in bowl_res:
                            st.markdown("### 🎯 Bowling Stats")
                            bowl_headers = bowl_res["headers"][1:]  # Skip ROWHEADER
                            bowl_data = {row["values"][0]: row["values"][1:] for row in bowl_res["values"]}
                            df_bowl = pd.DataFrame(bowl_data, index=bowl_headers).T
                            st.dataframe(df_bowl)

                    st.divider()
        else:
            st.warning("⚠️ No players found!")

    except Exception as e:
        st.error(f"Error fetching data: {e}")


# SQL

In [ ]:
import streamlit as st
import pymysql
import pandas as pd
from datetime import datetime, timedelta

# -----------------------------
# Database connection (single instance, reused)
# -----------------------------
try:
    conn = pymysql.connect(
        host="127.0.0.1",
        user="cricapp",
        password="Strong!Pass#123",
        database="Cricbuzz_project",
        port=3306
    )
except Exception as e:
    st.error(f"Database connection error: {e}")
    conn = None

# -----------------------------
# Streamlit app settings
# -----------------------------
st.set_page_config(page_title="Cricket Statistics Dashboard", layout="wide")
st.title("🏏 Cricket Statistics Dashboard")
st.markdown("---")

if conn:

    # -----------------------------
    # Question 1: Indian Players
    # -----------------------------
    st.header("1. Indian Cricket Players")
    try:
        query = """
        SELECT name as full_name, category as playing_role, 
               battingStyle as batting_style, bowlingStyle as bowling_style
        FROM indian_players
        ORDER BY name
        """
        df1 = pd.read_sql(query, conn)
        st.dataframe(df1, use_container_width=True)
    except Exception as e:
        st.warning(f"No data found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 2: Recent Matches (last 30 days)
    # -----------------------------
    st.header("2. Recent Matches (Last 30 Days)")
    try:
        thirty_days_ago = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
        query = """
        SELECT matchDesc as match_description, team1Name as team1, 
               team2Name as team2, venueGround as venue, startDate as match_date
        FROM match_results
        WHERE startDate >= %s
        ORDER BY startDate DESC
        """
        df2 = pd.read_sql(query, conn, params=[thirty_days_ago])
        st.dataframe(df2, use_container_width=True)
    except Exception as e:
        st.warning(f"No recent matches found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 3: Top 10 ODI run scorers
    # -----------------------------
    st.header("3. Top 10 ODI Run Scorers")
    try:
        query = """
        SELECT Player as player_name, Runs as total_runs, 
               Average as batting_average
        FROM top_odi_runs
        ORDER BY Runs DESC
        LIMIT 10
        """
        df3 = pd.read_sql(query, conn)
        st.dataframe(df3, use_container_width=True)
    except Exception as e:
        st.warning(f"No ODI scorers found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 4: Large capacity venues
    # -----------------------------
    st.header("4. Large Capacity Venues")
    try:
        query = """
        SELECT ground as venue_name, city, country, capacity
        FROM venues
        WHERE capacity > 50000
        ORDER BY capacity DESC
        """
        df4 = pd.read_sql(query, conn)
        st.dataframe(df4, use_container_width=True)
    except Exception as e:
        st.warning(f"No large venues found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 5: Team wins
    # -----------------------------
    st.header("5. Matches Won by Each Team")
    try:
        query = """
        SELECT 
            t.team_id,
            t.team_name,
            COUNT(*) AS total_wins
        FROM test_teams t
        JOIN matches m ON t.team_id = m.match_winner_id
        WHERE m.match_winner_id IS NOT NULL 
          AND m.match_winner_id != 'draw'
        GROUP BY t.team_id, t.team_name
        ORDER BY total_wins DESC;
        """
        df5 = pd.read_sql(query, conn)
        st.dataframe(df5, use_container_width=True)
    except Exception as e:
        st.warning(f"No data found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 6: Players by role
    # -----------------------------
    st.header("6. Players by Playing Role")
    try:
        query = """
        SELECT role, COUNT(*) as player_count
        FROM teams_players
        WHERE role IS NOT NULL AND role != ''
        GROUP BY role
        ORDER BY player_count DESC
        """
        df6 = pd.read_sql(query, conn)
        st.dataframe(df6, use_container_width=True)
    except Exception as e:
        st.warning(f"No role data. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 7: Highest scores by format
    # -----------------------------
    st.header("7. Highest Individual Scores by Format")
    try:
        query = """
        SELECT Format as format, MAX(Highest_Score) as highest_score
        FROM highest_scores
        GROUP BY Format
        ORDER BY highest_score DESC
        """
        df7 = pd.read_sql(query, conn)
        st.dataframe(df7, use_container_width=True)
    except Exception as e:
        st.warning(f"No highest score data. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 8: 2024 cricket series
    # -----------------------------
    st.header("8. 2024 Cricket Series")
    try:
        query = """
        SELECT series_name, host_country, match_type, 
               start_date, total_matches
        FROM cricket_series_2024
        WHERE YEAR(start_date) = 2024
        ORDER BY start_date DESC
        """
        df8 = pd.read_sql(query, conn)
        st.dataframe(df8, use_container_width=True)
    except Exception as e:
        st.warning(f"No 2024 series data. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 9: All-rounders
    # -----------------------------
    st.header("9. All-Rounder Players (1000+ Runs & 50+ Wickets)")
    try:
        query = """
        SELECT 
            playerName,
            (COALESCE(Runs_Test_bat,0) + COALESCE(Runs_ODI_bat,0) + COALESCE(Runs_T20_bat,0)) AS TotalRuns,
            (COALESCE(Wickets_Test_bowl,0) + COALESCE(Wickets_ODI_bowl,0) + COALESCE(Wickets_T20_bowl,0)) AS TotalWickets
        FROM players_with_stats
        WHERE (COALESCE(Runs_Test_bat,0) + COALESCE(Runs_ODI_bat,0) + COALESCE(Runs_T20_bat,0)) > 1000
        AND (COALESCE(Wickets_Test_bowl,0) + COALESCE(Wickets_ODI_bowl,0) + COALESCE(Wickets_T20_bowl,0)) > 50
        ORDER BY TotalRuns DESC, TotalWickets DESC
        """
        df9 = pd.read_sql(query, conn)
        st.dataframe(df9, use_container_width=True)
    except Exception as e:
        st.warning(f"No all-rounder data found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 10: Last 20 completed matches
    # -----------------------------
    st.header("10. Last 20 Completed Matches")
    try:
        query = """
        select * from recent_matches
        limit 20
        """
        df10 = pd.read_sql(query, conn)
        st.dataframe(df10, use_container_width=True)
    except Exception as e:
        st.warning(f"No completed matches found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 11: Compare players across formats
    # -----------------------------
    st.header("11. Player Performance Across Formats")
    try:
        query = """
        SELECT 
            playerName,
            Runs_Test_bat AS test_runs,
            Runs_ODI_bat AS odi_runs,
            Runs_T20_bat AS t20_runs,

            ROUND(
                (
                    (Runs_Test_bat + Runs_ODI_bat + Runs_T20_bat) * 1.0
                ) / NULLIF(
                    ((Innings_Test_bat - `Not Out_Test_bat`) +
                     (Innings_ODI_bat - `Not Out_ODI_bat`) +
                     (Innings_T20_bat - `Not Out_T20_bat`)), 0
                ), 2
            ) AS overall_batting_average

        FROM players_with_stats
        WHERE 
            (
              (CASE WHEN Runs_Test_bat > 0 THEN 1 ELSE 0 END) +
              (CASE WHEN Runs_ODI_bat > 0 THEN 1 ELSE 0 END) +
              (CASE WHEN Runs_T20_bat > 0 THEN 1 ELSE 0 END)
            ) >= 2
        ORDER BY overall_batting_average DESC, playerName
        """
        df11 = pd.read_sql(query, conn)
        st.dataframe(df11, use_container_width=True)
    except Exception as e:
        st.warning(f"No comparative data found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 12: Home vs Away Performance
    # -----------------------------
    st.header("12. Home vs Away Performance")
    try:
        query = """
        WITH team_countries AS (
            SELECT
                team_id,
                team_name,
                CASE
                    WHEN UPPER(team_name) LIKE '%ENGLAND%' THEN 'England'
                    WHEN UPPER(team_name) LIKE '%INDIA%' THEN 'India'
                    WHEN UPPER(team_name) LIKE '%AUSTRALIA%' THEN 'Australia'
                    WHEN UPPER(team_name) LIKE '%NEW ZEALAND%' THEN 'New Zealand'
                    WHEN UPPER(team_name) LIKE '%SOUTH AFRICA%' THEN 'South Africa'
                    WHEN UPPER(team_name) LIKE '%PAKISTAN%' THEN 'Pakistan'
                    WHEN UPPER(team_name) LIKE '%SRI LANKA%' THEN 'Sri Lanka'
                    WHEN UPPER(team_name) LIKE '%WEST INDIES%' THEN 'West Indies'
                    WHEN UPPER(team_name) LIKE '%BANGLADESH%' THEN 'Bangladesh'
                    WHEN UPPER(team_name) LIKE '%ZIMBABWE%' THEN 'Zimbabwe'
                    WHEN UPPER(team_name) LIKE '%AFGHANISTAN%' THEN 'Afghanistan'
                    WHEN UPPER(team_name) LIKE '%IRELAND%' THEN 'Ireland'
                    ELSE 'Other'
                END AS country
            FROM test_teams
        ),
        venue_countries AS (
            SELECT DISTINCT
                venue_city,
                CASE
                    WHEN UPPER(venue_city) LIKE '%LONDON%' OR UPPER(venue_city) LIKE '%NOTTINGHAM%' OR UPPER(venue_city) LIKE '%LEEDS%' THEN 'England'
                    WHEN UPPER(venue_city) LIKE '%SYDNEY%' OR UPPER(venue_city) LIKE '%MELBOURNE%' OR UPPER(venue_city) LIKE '%BRISBANE%' THEN 'Australia'
                    WHEN UPPER(venue_city) LIKE '%MUMBAI%' OR UPPER(venue_city) LIKE '%DELHI%' OR UPPER(venue_city) LIKE '%BANGALORE%' THEN 'India'
                    WHEN UPPER(venue_city) LIKE '%WELLINGTON%' OR UPPER(venue_city) LIKE '%AUCKLAND%' THEN 'New Zealand'
                    WHEN UPPER(venue_city) LIKE '%JOHANNESBURG%' OR UPPER(venue_city) LIKE '%CAPE TOWN%' THEN 'South Africa'
                    WHEN UPPER(venue_city) LIKE '%LAHORE%' OR UPPER(venue_city) LIKE '%KARACHI%' THEN 'Pakistan'
                    WHEN UPPER(venue_city) LIKE '%COLOMBO%' OR UPPER(venue_city) LIKE '%KANDY%' THEN 'Sri Lanka'
                    WHEN UPPER(venue_city) LIKE '%BRIDGETOWN%' OR UPPER(venue_city) LIKE '%KINGSTON%' THEN 'West Indies'
                    WHEN UPPER(venue_city) LIKE '%DHAKA%' OR UPPER(venue_city) LIKE '%CHITTAGONG%' THEN 'Bangladesh'
                    WHEN UPPER(venue_city) LIKE '%HARARE%' THEN 'Zimbabwe'
                    ELSE 'Neutral'
                END AS country
            FROM matches
            WHERE venue_city IS NOT NULL
        ),
        match_venues AS (
            SELECT
                m.match_id,
                m.team1_id,
                m.team2_id,
                m.match_winner_id,
                vc.country AS venue_country,
                tc1.country AS team1_country,
                tc2.country AS team2_country
            FROM matches m
            JOIN venue_countries vc ON m.venue_city = vc.venue_city
            JOIN team_countries tc1 ON m.team1_id = tc1.team_id
            JOIN team_countries tc2 ON m.team2_id = tc2.team_id
            WHERE m.match_winner_id IS NOT NULL AND m.match_winner_id != 'draw'
        ),
        team_performance AS (
            SELECT
                t.team_id,
                t.team_name,
                COUNT(CASE WHEN mv.venue_country = tc.country THEN 1 END) AS home_matches,
                COUNT(CASE WHEN mv.venue_country = tc.country AND mv.match_winner_id = t.team_id THEN 1 END) AS home_wins,
                COUNT(CASE WHEN mv.venue_country != tc.country AND mv.venue_country != 'Neutral' THEN 1 END) AS away_matches,
                COUNT(CASE WHEN mv.venue_country != tc.country AND mv.venue_country != 'Neutral' AND mv.match_winner_id = t.team_id THEN 1 END) AS away_wins,
                COUNT(CASE WHEN mv.venue_country = 'Neutral' THEN 1 END) AS neutral_matches,
                COUNT(CASE WHEN mv.venue_country = 'Neutral' AND mv.match_winner_id = t.team_id THEN 1 END) AS neutral_wins
            FROM test_teams t
            JOIN team_countries tc ON t.team_id = tc.team_id
            JOIN match_venues mv ON t.team_id IN (mv.team1_id, mv.team2_id)
            GROUP BY t.team_id, t.team_name
        )
        SELECT
            team_name,
            home_matches,
            home_wins,
            ROUND(home_wins * 100.0 / NULLIF(home_matches, 0), 2) AS home_win_percentage,
            away_matches,
            away_wins,
            ROUND(away_wins * 100.0 / NULLIF(away_matches, 0), 2) AS away_win_percentage,
            neutral_matches,
            neutral_wins,
            ROUND(neutral_wins * 100.0 / NULLIF(neutral_matches, 0), 2) AS neutral_win_percentage,
            (home_matches + away_matches + neutral_matches) AS total_matches
        FROM team_performance
        WHERE home_matches + away_matches >= 5
        ORDER BY home_win_percentage DESC;
        """
        df12 = pd.read_sql(query, conn)
        st.dataframe(df12, use_container_width=True)
    except Exception as e:
        st.warning(f"No data found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 17: Toss Advantage Analysis
    # -----------------------------
    st.header("17. Toss Advantage Analysis")
    try:
        query = """
        WITH toss_analysis AS (
            SELECT
                m.match_format,
                COUNT(*) AS total_matches,
                COUNT(CASE WHEN m.match_winner_id = m.toss_winner_id THEN 1 END) AS toss_winner_wins,
                COUNT(CASE WHEN m.toss_winner_id = m.team1_id THEN 1 END) AS toss_bat_first,
                COUNT(CASE WHEN m.toss_winner_id = m.team1_id AND m.match_winner_id = m.toss_winner_id THEN 1 END) AS toss_bat_win,
                COUNT(CASE WHEN m.toss_winner_id = m.team2_id THEN 1 END) AS toss_field_first,
                COUNT(CASE WHEN m.toss_winner_id = m.team2_id AND m.match_winner_id = m.toss_winner_id THEN 1 END) AS toss_field_win
            FROM matches m
            WHERE m.match_winner_id IS NOT NULL AND m.match_winner_id != 'draw'
            GROUP BY m.match_format
        )
        SELECT
            match_format,
            total_matches,
            toss_winner_wins,
            ROUND(toss_winner_wins * 100.0 / total_matches, 2) AS toss_win_percentage,
            toss_bat_first,
            toss_bat_win,
            ROUND(toss_bat_win * 100.0 / NULLIF(toss_bat_first, 0), 2) AS bat_first_win_percentage,
            toss_field_first,
            toss_field_win,
            ROUND(toss_field_win * 100.0 / NULLIF(toss_field_first, 0), 2) AS field_first_win_percentage
        FROM toss_analysis
        ORDER BY match_format;
        """
        df17 = pd.read_sql(query, conn)
        st.dataframe(df17, use_container_width=True)
    except Exception as e:
        st.warning(f"No data found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 18: Most Economical Bowlers (ODI & T20)
    # -----------------------------
    st.header("18. Most Economical Bowlers (ODI & T20)")
    try:
        query = """
        SELECT 
            playerName,
           (COALESCE(Wickets_ODI_bowl,0) + COALESCE(Wickets_T20_bowl,0)) AS total_wickets,
            ROUND(
                (COALESCE(Runs_ODI_bowl,0) + COALESCE(Runs_T20_bowl,0)) * 6.0  
            / NULLIF((COALESCE(Balls_ODI_bowl,0) + COALESCE(Balls_T20_bowl,0)), 0), 2
            ) AS overall_economy
            FROM players_with_stats
        WHERE 
            -- Played at least 10 matches in LO formats
            (COALESCE(Matches_ODI_bowl,0) + COALESCE(Matches_T20_bowl,0)) >= 10
            -- Bowled at least 2 overs per match on average
        AND ( (COALESCE(Balls_ODI_bowl,0) + COALESCE(Balls_T20_bowl,0))  
              / NULLIF((COALESCE(Matches_ODI_bowl,0) + COALESCE(Matches_T20_bowl,0)), 0)  
            ) >= 12
        ORDER BY overall_economy ASC, total_wickets DESC, playerName
        """
        df18 = pd.read_sql(query, conn)
        st.dataframe(df18, use_container_width=True)
        
    except Exception as e:
        st.warning(f"No economical bowlers found. Error: {e}")
    st.markdown("---")

    # -----------------------------
    # Question 20: Player Matches & Batting Averages Across Formats
    # -----------------------------
    st.header("20. Player Matches & Batting Averages Across Formats")
    try:
        query = """
        SELECT
            playerName,
            Matches_Test_bat AS Test_Matches,
            Average_Test_bat AS Test_Batting_Avg,
            Matches_ODI_bat AS ODI_Matches,
            Average_ODI_bat AS ODI_Batting_Avg,
            Matches_T20_bat AS T20_Matches,
            Average_T20_bat AS T20_Batting_Avg,
            (COALESCE(Matches_Test_bat,0) + COALESCE(Matches_ODI_bat,0) + COALESCE(Matches_T20_bat,0)) AS Total_Matches
        FROM
            players_with_stats
        WHERE
            (COALESCE(Matches_Test_bat,0) + COALESCE(Matches_ODI_bat,0) + COALESCE(Matches_T20_bat,0)) >= 20
        ORDER BY
            Total_Matches DESC, playerName
        """
        df20 = pd.read_sql(query, conn)
        st.dataframe(df20, use_container_width=True)
    
    except Exception as e:
        st.warning(f"No players found with at least 20 matches. Error: {e}")
    st.markdown("---")


    # -----------------------------
    # Question 21: Comprehensive Player Performance Ranking
    # -----------------------------
    st.header("21. Comprehensive Player Performance Ranking without fielding")
    try:
        
        query = """
        WITH test_rank AS (
            SELECT 
                playerName AS test_player,
                ROUND((
                    (COALESCE(Runs_Test_bat,0) * 0.01) +
                    (COALESCE(Average_Test_bat,0) * 0.5) +
                    (COALESCE(SR_Test_bat,0) * 0.3) +
                    (COALESCE(Wickets_Test_bowl,0) * 2) +
                    ((50 - COALESCE(Avg_Test_bowl,50)) * 0.5) +
                    ((6 - COALESCE(Eco_Test_bowl,6)) * 2)
                ),2) AS test_score,
                RANK() OVER (ORDER BY 
                    ((COALESCE(Runs_Test_bat,0) * 0.01) +
                     (COALESCE(Average_Test_bat,0) * 0.5) +
                     (COALESCE(SR_Test_bat,0) * 0.3) +
                     (COALESCE(Wickets_Test_bowl,0) * 2) +
                     ((50 - COALESCE(Avg_Test_bowl,50)) * 0.5) +
                     ((6 - COALESCE(Eco_Test_bowl,6)) * 2)) DESC
                ) AS rnk
            FROM players_with_stats
        ),
        odi_rank AS (
            SELECT 
                playerName AS odi_player,
                ROUND((
                    (COALESCE(Runs_ODI_bat,0) * 0.01) +
                    (COALESCE(Average_ODI_bat,0) * 0.5) +
                    (COALESCE(SR_ODI_bat,0) * 0.3) +
                    (COALESCE(Wickets_ODI_bowl,0) * 2) +
                    ((50 - COALESCE(Avg_ODI_bowl,50)) * 0.5) +
                    ((6 - COALESCE(Eco_ODI_bowl,6)) * 2)
                ),2) AS odi_score,
                RANK() OVER (ORDER BY 
                    ((COALESCE(Runs_ODI_bat,0) * 0.01) +
                     (COALESCE(Average_ODI_bat,0) * 0.5) +
                     (COALESCE(SR_ODI_bat,0) * 0.3) +
                     (COALESCE(Wickets_ODI_bowl,0) * 2) +
                     ((50 - COALESCE(Avg_ODI_bowl,50)) * 0.5) +
                     ((6 - COALESCE(Eco_ODI_bowl,6)) * 2)) DESC
                ) AS rnk
            FROM players_with_stats
        ),
        t20_rank AS (
            SELECT 
                playerName AS t20_player,
                ROUND((
                    (COALESCE(Runs_T20_bat,0) * 0.01) +
                    (COALESCE(Average_T20_bat,0) * 0.5) +
                    (COALESCE(SR_T20_bat,0) * 0.3) +
                    (COALESCE(Wickets_T20_bowl,0) * 2) +
                    ((50 - COALESCE(Avg_T20_bowl,50)) * 0.5) +
                    ((6 - COALESCE(Eco_T20_bowl,6)) * 2)
                ),2) AS t20_score,
                RANK() OVER (ORDER BY 
                    ((COALESCE(Runs_T20_bat,0) * 0.01) +
                     (COALESCE(Average_T20_bat,0) * 0.5) +
                     (COALESCE(SR_T20_bat,0) * 0.3) +
                     (COALESCE(Wickets_T20_bowl,0) * 2) +
                     ((50 - COALESCE(Avg_T20_bowl,50)) * 0.5) +
                     ((6 - COALESCE(Eco_T20_bowl,6)) * 2)) DESC
                ) AS rnk
            FROM players_with_stats
        )
        SELECT 
            t.test_score, t.test_player,
            o.odi_score,  o.odi_player,
            tt.t20_score, tt.t20_player
        FROM test_rank t
        JOIN odi_rank o ON t.rnk = o.rnk
        JOIN t20_rank tt ON t.rnk = tt.rnk
        ORDER BY t.rnk
        LIMIT 20;
        """
        df21 = pd.read_sql(query, conn)
        st.dataframe(df21, use_container_width=True)
    except Exception as e:
        st.warning(f"No performance ranking found. Error: {e}")
    st.markdown("---")

    st.markdown("*Cricket Statistics Dashboard - Created with Streamlit*")

    # -----------------------------
    # Question 22: Head-to-Head Analysis
    # -----------------------------
    st.header("22. Head-to-Head Analysis")
    try:
        query = """
        WITH team_pairs AS (
            SELECT
                LEAST(m.team1_id, m.team2_id) AS team_a_id,
                GREATEST(m.team1_id, m.team2_id) AS team_b_id,
                t1.team_name AS team_a_name,
                t2.team_name AS team_b_name,
                COUNT(*) AS total_matches,
                COUNT(CASE WHEN m.match_winner_id = LEAST(m.team1_id, m.team2_id) THEN 1 END) AS team_a_wins,
                COUNT(CASE WHEN m.match_winner_id = GREATEST(m.team1_id, m.team2_id) THEN 1 END) AS team_b_wins,
                COUNT(CASE WHEN m.match_winner_id = 'draw' THEN 1 END) AS draws,
                AVG(CASE WHEN m.match_winner_id = LEAST(m.team1_id, m.team2_id) THEN m.margin_runs END) AS avg_win_margin_runs_a,
                AVG(CASE WHEN m.match_winner_id = LEAST(m.team1_id, m.team2_id) THEN m.margin_wickets END) AS avg_win_margin_wickets_a,
                AVG(CASE WHEN m.match_winner_id = GREATEST(m.team1_id, m.team2_id) THEN m.margin_runs END) AS avg_win_margin_runs_b,
                AVG(CASE WHEN m.match_winner_id = GREATEST(m.team1_id, m.team2_id) THEN m.margin_wickets END) AS avg_win_margin_wickets_b
            FROM matches m
            JOIN test_teams t1 ON LEAST(m.team1_id, m.team2_id) = t1.team_id
            JOIN test_teams t2 ON GREATEST(m.team1_id, m.team2_id) = t2.team_id
            WHERE m.match_winner_id IS NOT NULL
            GROUP BY LEAST(m.team1_id, m.team2_id), GREATEST(m.team1_id, m.team2_id), t1.team_name, t2.team_name
            HAVING COUNT(*) >= 5
        )
        SELECT
            team_a_name,
            team_b_name,
            total_matches,
            team_a_wins,
            team_b_wins,
            draws,
            ROUND(team_a_wins * 100.0 / total_matches, 2) AS team_a_win_percentage,
            ROUND(team_b_wins * 100.0 / total_matches, 2) AS team_b_win_percentage,
            ROUND(COALESCE(avg_win_margin_runs_a, 0), 1) AS avg_win_margin_runs_a,
            ROUND(COALESCE(avg_win_margin_wickets_a, 0), 1) AS avg_win_margin_wickets_a,
            ROUND(COALESCE(avg_win_margin_runs_b, 0), 1) AS avg_win_margin_runs_b,
            ROUND(COALESCE(avg_win_margin_wickets_b, 0), 1) AS avg_win_margin_wickets_b
        FROM team_pairs
        ORDER BY total_matches DESC;
        """
        df22 = pd.read_sql(query, conn)
        st.dataframe(df22, use_container_width=True)


    except Exception as e:
        st.warning(f"No data found. Error: {e}")
    st.markdown("---")


else:
    st.stop()


# CRUD

In [ ]:
import streamlit as st
import pymysql
import pandas as pd
from datetime import datetime
import time

# Database connection function
def get_connection():
    try:
        conn = pymysql.connect(
            host="127.0.0.1",
            user="cricapp",
            password="Strong!Pass#123",
            database="Cricbuzz_project",
            port=3306,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        return conn
    except Exception as e:
        st.error(f"Database connection failed: {e}")
        return None

# CRUD operations class
class CricketCRUD:
    def __init__(self):
        self.conn = get_connection()
    
    def execute_query(self, query, params=None, fetch=True):
        try:
            with self.conn.cursor() as cursor:
                cursor.execute(query, params)
                if fetch:
                    result = cursor.fetchall()
                    self.conn.commit()
                    return result
                else:
                    self.conn.commit()
                    return cursor.rowcount
        except Exception as e:
            st.error(f"Query execution failed: {e}")
            return None
    
    def get_table_data(self, table_name, where_clause=""):
        query = f"SELECT * FROM `{table_name}` {where_clause} LIMIT 1000"
        return self.execute_query(query)
    
    def get_highest_scores_by_format(self, format_type):
        query = "SELECT * FROM `highest_scores` WHERE Format = %s"
        return self.execute_query(query, (format_type,))
    
    def get_player_by_id(self, player_id):
        query = "SELECT * FROM `players_with_stats` WHERE playerId = %s"
        return self.execute_query(query, (player_id,))
    
    def insert_data(self, table_name, data):
        columns = ', '.join([f"`{key}`" for key in data.keys()])
        placeholders = ', '.join(['%s'] * len(data))
        query = f"INSERT INTO `{table_name}` ({columns}) VALUES ({placeholders})"
        return self.execute_query(query, tuple(data.values()), fetch=False)
    
    def update_data(self, table_name, primary_key, pk_value, data):
        set_clause = ', '.join([f"`{key}` = %s" for key in data.keys()])
        query = f"UPDATE `{table_name}` SET {set_clause} WHERE `{primary_key}` = %s"
        params = tuple(data.values()) + (pk_value,)
        return self.execute_query(query, params, fetch=False)
    
    def delete_data(self, table_name, primary_key, pk_value):
        query = f"DELETE FROM `{table_name}` WHERE `{primary_key}` = %s"
        return self.execute_query(query, (pk_value,), fetch=False)
    
    def get_all_player_ids(self):
        query = "SELECT playerId, playerName FROM `players_with_stats` ORDER BY playerId LIMIT 1000"
        return self.execute_query(query)
    
    def close(self):
        if self.conn:
            self.conn.close()

# Helper function to get primary key column
def get_primary_key_column(df):
    for col in df.columns:
        if any(keyword in col.lower() for keyword in ['id', 'pk', 'primary']):
            return col
    return df.columns[0] if not df.empty else None

# Main application
def main():
    st.set_page_config(page_title="Cricket Database CRUD", layout="wide")
    st.title("🏏 Cricket Database Management System")
    
    # Initialize CRUD instance
    crud = CricketCRUD()
    if not crud.conn:
        st.error("Cannot connect to database. Please check your connection settings.")
        return
    
    try:
        # Sidebar for table selection

        # Sidebar for table selection
        tables = [
            "cricket_series_2024", "highest_scores", "indian_players", 
            "international_teams", "match_results", "players_with_stats",
            "recent_matches", "teams_players", "top_odi_runs", 
            "top_test_runs", "venues",
            # Newly added tables
            "matches", "test_teams", "innings"
        ]
        
        selected_table = st.sidebar.selectbox("Select Table", tables)
        operation = st.sidebar.radio("Operation", ["View", "Create", "Update", "Delete"])

        
        st.header(f"{operation} - {selected_table}")
        
        # Special handling for highest_scores table
        if selected_table == "highest_scores" and operation == "View":
            st.subheader("Search by Format")
            format_type = st.selectbox("Select Format", ["Test", "ODI", "T20"], index=0)
            
            if st.button("Search by Format"):
                data = crud.get_highest_scores_by_format(format_type)
                if data:
                    df = pd.DataFrame(data)
                    st.dataframe(df, use_container_width=True)
                    st.metric(f"Total {format_type} Records", len(df))
                else:
                    st.warning(f"No records found for {format_type} format")
            
            # Also show all data option
            if st.checkbox("Show all highest scores"):
                data = crud.get_table_data(selected_table)
                if data:
                    df = pd.DataFrame(data)
                    st.dataframe(df, use_container_width=True)
                    st.metric("Total Records", len(df))
        
        # Special handling for players_with_stats in Update operation
        elif selected_table == "players_with_stats" and operation == "Update":
            st.subheader("Search Player by ID")
            
            # First, get all player IDs for selection
            all_players = crud.get_all_player_ids()
            if all_players:
                # Create a mapping of playerId to playerName for display
                player_options = {f"{player['playerId']} - {player['playerName']}": player['playerId'] for player in all_players}
                selected_display = st.selectbox("Select Player", list(player_options.keys()))
                selected_player_id = player_options[selected_display]
                
                if st.button("Load Player Data"):
                    player_data = crud.get_player_by_id(selected_player_id)
                    if player_data:
                        df = pd.DataFrame(player_data)
                        st.success(f"Loaded data for Player ID: {selected_player_id}")
                        
                        # Display current player info
                        player = player_data[0]
                        col1, col2, col3 = st.columns(3)
                        col1.metric("Player Name", player['playerName'])
                        col2.metric("Team", player['teamName'])
                        col3.metric("Country", player['countryName'])
                        
                        st.subheader("Update Player Statistics")
                        
                        # Group fields by category for better organization
                        batting_stats = {}
                        bowling_stats = {}
                        personal_info = {}
                        
                        for col in df.columns:
                            if any(keyword in col.lower() for keyword in ['bat', 'runs', 'average', 'sr', '50s', '100s', 'innings', 'matches']):
                                batting_stats[col] = player[col]
                            elif any(keyword in col.lower() for keyword in ['bowl', 'wickets', 'eco', 'maiden', 'overs']):
                                bowling_stats[col] = player[col]
                            else:
                                personal_info[col] = player[col]
                        
                        update_fields = {}
                        
                        # Personal Info
                        with st.expander("Personal Information"):
                            for col, value in personal_info.items():
                                if col != 'playerId':
                                    if pd.api.types.is_numeric_dtype(df[col]) and 'id' not in col.lower():
                                        update_fields[col] = st.number_input(
                                            col, value=float(value) if value is not None else 0.0,
                                            key=f"personal_{col}"
                                        )
                                    else:
                                        update_fields[col] = st.text_input(
                                            col, value=str(value) if value is not None else "",
                                            key=f"personal_{col}"
                                        )
                        
                        # Batting Stats
                        with st.expander("Batting Statistics"):
                            for col, value in batting_stats.items():
                                if pd.api.types.is_numeric_dtype(df[col]):
                                    update_fields[col] = st.number_input(
                                        col, value=float(value) if value is not None else 0.0,
                                        key=f"batting_{col}"
                                    )
                                else:
                                    update_fields[col] = st.text_input(
                                        col, value=str(value) if value is not None else "",
                                        key=f"batting_{col}"
                                    )
                        
                        # Bowling Stats
                        with st.expander("Bowling Statistics"):
                            for col, value in bowling_stats.items():
                                if pd.api.types.is_numeric_dtype(df[col]):
                                    update_fields[col] = st.number_input(
                                        col, value=float(value) if value is not None else 0.0,
                                        key=f"bowling_{col}"
                                    )
                                else:
                                    update_fields[col] = st.text_input(
                                        col, value=str(value) if value is not None else "",
                                        key=f"bowling_{col}"
                                    )
                        
                        if st.button("Update Player Record"):
                            # Remove None values from update_fields
                            update_fields = {k: v for k, v in update_fields.items() if v is not None}
                            if update_fields:
                                result = crud.update_data(selected_table, 'playerId', selected_player_id, update_fields)
                                if result:
                                    st.success("Player record updated successfully!")
                                    time.sleep(2)
                                    st.rerun()
                            else:
                                st.warning("No fields to update")
                    else:
                        st.error("Player not found!")
            else:
                st.warning("No players found in the database")
        
        # Default behavior for other tables
        else:
            # Get table data
            data = crud.get_table_data(selected_table)
            if data is None:
                st.error("Failed to fetch data")
                return
            
            if not data:
                st.warning("No data found in the table")
                return
                
            df = pd.DataFrame(data)
            
            if operation == "View":
                st.dataframe(df, use_container_width=True)
                
                # Show statistics
                st.subheader("Table Information")
                col1, col2, col3 = st.columns(3)
                col1.metric("Total Records", len(df))
                col2.metric("Columns", len(df.columns))
                col3.metric("Preview", f"{len(df)} rows")
                
            elif operation == "Create":
                st.subheader("Add New Record")
                
                # Get column information
                exclude_cols = ['created_at', 'updated_at']
                input_fields = {}
                
                for col in df.columns:
                    if not any(exclude in col.lower() for exclude in exclude_cols):
                        col_type = str(df[col].dtype)
                        
                        if 'int' in col_type:
                            input_fields[col] = st.number_input(col, value=0, key=f"create_{col}")
                        elif 'float' in col_type or 'decimal' in col_type:
                            input_fields[col] = st.number_input(col, value=0.0, key=f"create_{col}")
                        elif 'date' in col_type:
                            input_fields[col] = st.date_input(col, key=f"create_{col}")
                        elif 'datetime' in col_type:
                            input_fields[col] = st.datetime_input(col, key=f"create_{col}")
                        else:
                            input_fields[col] = st.text_input(col, key=f"create_{col}")
                
                if st.button("Add Record"):
                    if all(value is not None for value in input_fields.values()):
                        result = crud.insert_data(selected_table, input_fields)
                        if result:
                            st.success("Record added successfully!")
                            time.sleep(1)
                            st.rerun()
                    else:
                        st.error("Please fill all fields")
            
            elif operation == "Update" and selected_table != "players_with_stats":
                st.subheader("Update Record")
                
                if not df.empty:
                    primary_key_col = get_primary_key_column(df)
                    
                    if primary_key_col:
                        record_id = st.selectbox(f"Select {primary_key_col}", df[primary_key_col].tolist())
                        selected_record = df[df[primary_key_col] == record_id].iloc[0]
                        
                        st.write("Current values:")
                        st.json(selected_record.to_dict())
                        
                        # Input fields for update
                        update_fields = {}
                        for col in df.columns:
                            if col != primary_key_col and not any(exclude in col.lower() for exclude in ['created_at', 'updated_at']):
                                current_val = selected_record[col]
                                
                                if pd.api.types.is_numeric_dtype(df[col]):
                                    update_fields[col] = st.number_input(
                                        col, 
                                        value=float(current_val) if pd.notnull(current_val) else 0.0,
                                        key=f"update_{col}"
                                    )
                                elif pd.api.types.is_datetime64_any_dtype(df[col]):
                                    update_fields[col] = st.date_input(
                                        col, 
                                        value=current_val if pd.notnull(current_val) else datetime.now().date(),
                                        key=f"update_{col}"
                                    )
                                else:
                                    update_fields[col] = st.text_input(
                                        col, 
                                        value=str(current_val) if pd.notnull(current_val) else "",
                                        key=f"update_{col}"
                                    )
                        
                        if st.button("Update Record"):
                            result = crud.update_data(selected_table, primary_key_col, record_id, update_fields)
                            if result:
                                st.success("Record updated successfully!")
                                time.sleep(1)
                                st.rerun()
                    else:
                        st.error("Could not identify primary key column")
                else:
                    st.warning("No records to update")
            
            elif operation == "Delete":
                st.subheader("Delete Record")
                
                if not df.empty:
                    primary_key_col = get_primary_key_column(df)
                    
                    if primary_key_col:
                        record_id = st.selectbox(f"Select {primary_key_col} to delete", df[primary_key_col].tolist())
                        selected_record = df[df[primary_key_col] == record_id].iloc[0]
                        
                        st.write("Record to be deleted:")
                        st.json(selected_record.to_dict())
                        
                        if st.button("Delete Record", type="secondary"):
                            confirm = st.checkbox("I confirm I want to delete this record")
                            if confirm:
                                result = crud.delete_data(selected_table, primary_key_col, record_id)
                                if result:
                                    st.success("Record deleted successfully!")
                                    time.sleep(1)
                                    st.rerun()
                    else:
                        st.error("Could not identify primary key column")
                else:
                    st.warning("No records to delete")
    
    finally:
        # Ensure connection is closed
        crud.close()

# Run the application
if __name__ == "__main__":
    main()